# Taxi CrossValidation with GPU accelerating on XGBoost

In this notebook, we will show you how to levarage GPU to accelerate taxi CrossValidation on XGBoost to find out the best model given a group parameters.

## Import classes
First we need load some common classes that both GPU version and CPU version will use:

In [16]:
import ml.dmlc.xgboost4j.scala.spark.{XGBoostRegressionModel, XGBoostRegressor}
import org.apache.spark.ml.evaluation.{RegressionEvaluator}
import org.apache.spark.ml.tuning.{ParamGridBuilder,CrossValidator}
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{FloatType, IntegerType, StructField, StructType}

what is new to xgboost-spark users is rapids.GpuDataReader and **rapids.CrossValidator**

In [17]:
// import ml.dmlc.xgboost4j.scala.spark.rapids.CrossValidator

Syntax Error.: 

## Set dataset path

In [18]:
// You need to update them to your real paths! The input data files can be the output of taxi-etl jobs, or you can
// just use the provided sample datasets under datasets path. 
val dataRoot = sys.env.getOrElse("DATA_ROOT", "/data")
val trainParquetPath=dataRoot + "/taxi/parquet/train"
val evalParquetPath=dataRoot + "/taxi/parquet/eval"

dataRoot = /data
trainParquetPath = /data/taxi/parquet/train
evalParquetPath = /data/taxi/parquet/eval


/data/taxi/parquet/eval

# Set the schema of the dataset

In [19]:
val labelColName = "fare_amount"
val schema =
    StructType(Array(
      StructField("vendor_id", FloatType),
      StructField("passenger_count", FloatType),
      StructField("trip_distance", FloatType),
      StructField("pickup_longitude", FloatType),
      StructField("pickup_latitude", FloatType),
      StructField("rate_code", FloatType),
      StructField("store_and_fwd", FloatType),
      StructField("dropoff_longitude", FloatType),
      StructField("dropoff_latitude", FloatType),
      StructField(labelColName, FloatType),
      StructField("hour", FloatType),
      StructField("year", IntegerType),
      StructField("month", IntegerType),
      StructField("day", FloatType),
      StructField("day_of_week", FloatType),
      StructField("is_weekend", FloatType)
    ))

labelColName = fare_amount
schema = StructType(StructField(vendor_id,FloatType,true), StructField(passenger_count,FloatType,true), StructField(trip_distance,FloatType,true), StructField(pickup_longitude,FloatType,true), StructField(pickup_latitude,FloatType,true), StructField(rate_code,FloatType,true), StructField(store_and_fwd,FloatType,true), StructField(dropoff_longitude,FloatType,true), StructField(dropoff_latitude,FloatType,true), StructField(fare_amount,FloatType,true), StructField(hour,FloatType,true), StructField(year,IntegerType,true), StructField(month,IntegerType,true), StructField(day,FloatType,true), StructField(day_of_week,FloatType,true), StructField(is_weekend,FloatType,true))


StructType(StructField(vendor_id,FloatType,true), StructField(passenger_count,FloatType,true), StructField(trip_distance,FloatType,true), StructField(pickup_longitude,FloatType,true), StructField(pickup_latitude,FloatType,true), StructField(rate_code,FloatType,true), StructField(store_and_fwd,FloatType,true), StructField(dropoff_longitude,FloatType,true), StructField(dropoff_latitude,FloatType,true), StructField(fare_amount,FloatType,true), StructField(hour,FloatType,true), StructField(year,IntegerType,true), StructField(month,IntegerType,true), StructField(day,FloatType,true), StructField(day_of_week,FloatType,true), StructField(is_weekend,FloatType,true))

## Create a new spark session and load data
we must create a new spark session to continue all spark operations. It will also be used to initilize the `GpuDataReader` which is a data reader powered by GPU.

NOTE: in this notebook, we have uploaded dependency jars when installing toree kernel. If we don't upload them at installation time, we can also upload in notebook by [%AddJar magic](https://toree.incubator.apache.org/docs/current/user/faq/). However, there's one restriction for `%AddJar`: the jar uploaded can only be available when `AddJar` is called after a new spark session is created. We must use it as below:

```scala
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().appName("Taxi-GPU-CV").getOrCreate
%AddJar file:/data/libs/rapids-4-spark-XXX.jar
%AddJar file:/data/libs/xgboost4j-spark-gpu_2.12-XXX.jar
%AddJar file:/data/libs/xgboost4j-gpu_2.12-XXX.jar
// ...
```

In [20]:
val spark = SparkSession.builder().appName("taxi-gpu-cv").getOrCreate()
val trainDs = spark.read.parquet(trainParquetPath)

spark = org.apache.spark.sql.SparkSession@1b953a9c
trainDs = [vendor_id: int, passenger_count: int ... 15 more fields]


[vendor_id: int, passenger_count: int ... 15 more fields]

## Find out features to train

In [21]:
val featureNames = schema.filter(_.name != labelColName).map(_.name).toArray

featureNames = Array(vendor_id, passenger_count, trip_distance, pickup_longitude, pickup_latitude, rate_code, store_and_fwd, dropoff_longitude, dropoff_latitude, hour, year, month, day, day_of_week, is_weekend)


Array(vendor_id, passenger_count, trip_distance, pickup_longitude, pickup_latitude, rate_code, store_and_fwd, dropoff_longitude, dropoff_latitude, hour, year, month, day, day_of_week, is_weekend)

In [22]:
val regressorParam = Map(
    "num_round" -> 100,
    "tree_method" -> "hist",
    "device" -> "cuda",
    "num_workers" -> 1)

regressorParam = Map(num_round -> 100, tree_method -> hist, device -> cuda,  num_workers -> 1)


Map(num_round -> 100, tree_method -> hist, device -> cuda, num_workers -> 1)

## Construct CrossValidator

In [23]:
val regressor = new XGBoostRegressor(regressorParam)
    .setLabelCol(labelColName)
    .setFeaturesCol(featureNames)
val paramGrid = new ParamGridBuilder()
    .addGrid(regressor.maxDepth, Array(3, 10))
    .addGrid(regressor.eta, Array(0.2, 0.6))
    .build()
val evaluator = new RegressionEvaluator().setLabelCol(labelColName)
val cv = new CrossValidator()
    .setEstimator(regressor)
    .setEvaluator(evaluator)
    .setEstimatorParamMaps(paramGrid)
    .setNumFolds(3)

regressor = xgbr_1c1bd6fa3a5f
paramGrid = 
evaluator = RegressionEvaluator: uid=regEval_c7293a967512, metricName=rmse, throughOrigin=false
cv = cv_06528fc9d704


Array({
	xgbr_1c1bd6fa3a5f-eta: 0.2,
	xgbr_1c1bd6fa3a5f-maxDepth: 3
}, {
	xgbr_1c1bd6fa3a5f-eta: 0.6,
	xgbr_1c1bd6fa3a5f-maxDepth: 3
}, {
	xgbr_1c1bd6fa3a5f-eta: 0.2,
	xgbr_1c1bd6fa3a5f-maxDepth: 10
}, {
	xgbr_1c1bd6fa3a5f-eta: 0.6,
	xgbr_1c1bd6fa3a5f-maxDepth: 10
})


cv_06528fc9d704

## train with CrossValidator

In [24]:
val model = cv.fit(trainDs).bestModel.asInstanceOf[XGBoostRegressionModel]

Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.210, DMLC_TRACKER_PORT=36551, DMLC_NUM_WORKER=1}
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.210, DMLC_TRACKER_PORT=40153, DMLC_NUM_WORKER=1}
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.210, DMLC_TRACKER_PORT=46553, DMLC_NUM_WORKER=1}
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.210, DMLC_TRACKER_PORT=50795, DMLC_NUM_WORKER=1}
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.210, DMLC_TRACKER_PORT=44927, DMLC_NUM_WORKER=1}
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.210, DMLC_TRACKER_PORT=55309, DMLC_NUM_WORKER=1}
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.210, DMLC_TRACKER_PORT=55163, DMLC_NUM_WORKER=1}
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.210, DMLC_TRACKER_PORT=54783, DMLC_NUM_WORKER=1}
Tracker started, with en

model = xgbr_1c1bd6fa3a5f


xgbr_1c1bd6fa3a5f

## tranform with best model trained by CrossValidator

In [25]:
val transformDs = spark.read.parquet(evalParquetPath)
val df = model.transform(transformDs).cache()
df.select("fare_amount", "prediction").show(5)

transformDs = [vendor_id: int, passenger_count: int ... 15 more fields]
df = [vendor_id: int, passenger_count: int ... 16 more fields]


+-----------+------------------+
|fare_amount|        prediction|
+-----------+------------------+
|       11.4|12.278875350952148|
|        7.4|7.4439215660095215|
|        5.0| 4.565710067749023|
|        8.5| 9.188780784606934|
|        7.4| 7.266360759735107|
+-----------+------------------+
only showing top 5 rows



[vendor_id: int, passenger_count: int ... 16 more fields]

In [26]:
val evaluator = new RegressionEvaluator().setLabelCol(labelColName)
val rmse = evaluator.evaluate(df)

evaluator = RegressionEvaluator: uid=regEval_1c57378a8fe1, metricName=rmse, throughOrigin=false
rmse = 2.2492672858545992


2.2492672858545992

In [27]:
spark.close()